In [2]:
from uuid import uuid4

from confluent_kafka import Consumer
from confluent_kafka.serialization import StringDeserializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer

In [7]:
def get_schema_from_schema_registry(schema_registry_url, schema_registry_subject):
    sr = SchemaRegistryClient({'url': schema_registry_url})
    latest_version = sr.get_latest_version(schema_registry_subject)

    return sr, latest_version

In [1]:
bootstrap_servers = "kafka-1:19092,kafka-2:19093"
topic = "test-avro-topic"
schema_registry_subject = topic + "-value"

In [4]:
schema_registry_url = 'http://schema-registry-1:8081'

schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [8]:
sr, latest_version = get_schema_from_schema_registry(schema_registry_url, schema_registry_subject)
schema_str = latest_version.schema.schema_str

In [9]:
avro_deserializer = AvroDeserializer(schema_registry_client,
                                         schema_str)

consumer_conf = {'bootstrap.servers': bootstrap_servers,
                     'group.id': 'python-cg',
                     'auto.offset.reset': "earliest"}

consumer = Consumer(consumer_conf)
consumer.subscribe([topic])

In [ ]:
while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            continue

        person = avro_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
        if person is not None: 
            print (person)
    except KeyboardInterrupt: 
        break        
consumer.close() 

{'id': '1001', 'firstName': 'Peter', 'lastName': 'Muster'}
{'id': '1001', 'firstName': 'Peter', 'lastName': 'Muster'}
